In [1]:
import random
from collections import defaultdict
from sentence_transformers import SentenceTransformer, SentencesDataset, models, losses, InputExample
from sentence_transformers.losses import TripletLoss
from sentence_transformers.readers import LabelSentenceReader, InputExample
from torch.utils.data import DataLoader
import os
import sys
import numpy as np
import json
import pandas as pd
from sentence_transformers import datasets
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import torch
from scipy.spatial import distance
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

In [4]:
fb = pd.read_csv('data/Covid/pre_existing_covid.csv') 
fb

,id,ad_creative_body,pred_theme,pred_phrase,dist_p,funding_entity,neg_theme,neg_phrase
0,322514789813495,U.S. Rep. Chip Roy (R-Texas) is leading effort...,VaccineMandate,Firing over vaccine mandates is oppression,0.325048,"Americans for Limited Government, Inc.",CovidPlan,Expand mask and PPE supply
1,253885516906049,The best part of being a kid is imagining big ...,VaccineRollout,FDA authorized covid vaccine for children ages...,0.466167,Children's Health System of Texas,VaccineReligion,I put it all in God's hands
2,263521289190020,The COVID-19 vaccines not only help protect yo...,VaccineRollout,CDC recommends vaccine for kids,0.358542,Children's Health System of Texas,CovidPlan,Support small business
3,311184624320430,Being your kid’s biggest fan means making sure...,VaccineRollout,CDC recommends vaccine for kids,0.421162,Children's Health System of Texas,VaccineMandate,I won't have the government tell me what to do
4,320222096723571,Looking out for your kids means looking out fo...,VaccineRollout,CDC recommends vaccine for kids,0.328104,Children's Health System of Texas,GovDistrust,Biden is a failure
...,...,...,...,...,...,...,...,...
9915,764566357481983,Gov. Sisolak to hold news conference on Nevada...,VaccineStatus,Covid vaccine information is here,0.547026,NaN,EncourageVaccination,Get boosted
9916,436102037557280,Researchers say that people with diabetes infe...,VaccineEfficacy,Unvaccinated people are dying at a rapid rate ...,0.438287,NaN,CovidPlan,Expand mask and PPE supply
9917,712359999420062,West Hartford News: State Outlines Roll-Out of...,VaccineStatus,Covid vaccine information is here,0.379729,NaN,CovidPlan,Support small business
9918,293902915337320,JESUS: Their wicked plans also include a new g...,VaccineReligion,Jesus is trying to protect us from covid by di...,0.394629,NaN,EncourageVaccination,Get boosted


In [58]:
thm = fb.loc[(fb.pred_theme == 'GovDistrust')]
#thm = fb.loc[(fb.pred_theme == 'GovTrust')]
# thm = fb.loc[(fb.pred_theme == 'VaccineRollout')]
# thm = fb.loc[(fb.pred_theme == 'VaccineSymptom')]
# thm = fb.loc[(fb.pred_theme == 'VaccineEquity')]
# thm = fb.loc[(fb.pred_theme == 'VaccineStatus')]
# thm = fb.loc[(fb.pred_theme == 'EncourageVaccination')]
# thm = fb.loc[(fb.pred_theme == 'VaccineMandate')]
# thm = fb.loc[(fb.pred_theme == 'VaccineReligion')]
# thm = fb.loc[(fb.pred_theme == 'VaccineEfficacy')]
# thm = fb.loc[(fb.pred_theme == 'VaccineDevelopment')]
# thm = fb.loc[(fb.pred_theme == 'CovidPlan')]
# thm = fb.loc[(fb.pred_theme == 'VaccineMisinformation')]
# thm = fb.loc[(fb.pred_theme == 'NaturalImmunity')]
#thm = fb.loc[(fb.pred_theme == 'Vote')]



thm = thm.reset_index(drop=True)
thm

,id,ad_creative_body,pred_theme,pred_phrase,dist_p,funding_entity,neg_theme,neg_phrase
0,513254653437798,Join us every Thursday for the Freedom Corner ...,Vote,Vote today,0.724631,Dana Lamar Criswell,NaturalImmunity,Natural immunity has higher level of protectio...
1,488070486162981,Take Our Poll Now! ⬇️⬇️,Vote,Vote today,0.579475,Healthy 603 Coalition,NaturalImmunity,Natural immunity would require a lot of people...
2,1076981432874064,🚨25 reasons why I am asking for a promotion:\n...,Vote,Please vote,0.652736,Anil Roodal Beephan,VaccineReligion,Disregard for others' lives isn't part of your...
3,911094039606655,The main difference between the two Wise Count...,Vote,Your vote matters,0.676259,Chad L Miller for Wise County Judge,CovidPlan,Expand mask and PPE supply
4,335116701953473,We are listening to South Carolina and Putting...,Vote,Go vote,0.638628,Barton for Congress,VaccineMandate,Airlines require vaccine passport
...,...,...,...,...,...,...,...,...
421,478143216705066,New York Governor Andrew Cuomo said on Monday ...,Vote,Go vote,0.806225,Stansberry Research,VaccineReligion,Disregard for others' lives isn't part of your...
422,510441487012789,"Vote April 6th, 2021.",Vote,Vote today,0.415573,Mulliner 4 Mayor,VaccineMisinformation,Fauci tortures dogs and puppies
423,944088713085362,The pressure continues to build on Governor An...,Vote,Go vote,0.677773,Colin D Moore,VaccineReligion,Disregard for others' lives isn't part of your...
424,429259541543534,Guy Molyneux of Hart Research shares what he f...,Vote,Your vote matters,0.675823,AARP,VaccineEfficacy,"Vaccines do work, ask a doctor or consult with..."


In [59]:
thm_sort = thm.sort_values(by=['dist_p'])
#thm_sort.head(10)

In [60]:
thm_sort.head(20).to_csv("data/Covid/GovDistrust_thm_iter1.csv")